In [304]:
# Import Dependencies
%matplotlib inline

# data analysis and wrangling
import math, time, random, datetime

# Data Manipulation
import numpy as np
import pandas as pd

# Visualization 
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')                        # The default theme is DarkGrid Theme .........

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


Importing the dataset: 

In [305]:
df = pd.read_csv('Downloads/Card_Attrition_Challenge_DT0.csv', sep=";")
df_test = pd.read_csv('Downloads/Card_Attrition_Challenge_DT0_Test.csv', sep=";")

Checking for duplicate values: 

In [306]:
sum(df.duplicated())

0

In [307]:
sum(df_test.duplicated())

0

Hence there are no duplicate values in our database... 

In [308]:
print(df.isnull().any())
print('*'*40)
print(df_test.isnull().any())

CLIENTNUM                   False
Attrition_Flag              False
Customer_Age                 True
Gender                      False
Dependent_count             False
Education_Level             False
Marital_Status              False
Income_Category             False
Card_Category               False
Months_on_book              False
Total_Relationship_Count    False
Months_Inactive_12_mon      False
Contacts_Count_12_mon       False
Credit_Limit                 True
Total_Revolving_Bal         False
Trans_Amt_Oct12             False
Trans_Amt_Nov12             False
Trans_Amt_Dec12             False
Trans_Amt_Jan13             False
Trans_Amt_Feb13             False
Trans_Amt_Mar13             False
Trans_Count_Oct12           False
Trans_Count_Nov12           False
Trans_Count_Dec12           False
Trans_Count_Jan13           False
Trans_Count_Feb13           False
Trans_Count_Mar13           False
dtype: bool
****************************************
CLIENTNUM                   F

So there are missing values in 2 features, in Customer_Age and another in Credit_Limit.....

In [309]:
print(df.describe(include=['O']))
print('*'*40)
print(df_test.describe(include=['O']))

       Gender Education_Level Marital_Status Income_Category Card_Category
count    7498            7498           7498            7498          7498
unique      2               7              4               6             4
top         F        Graduate        Married  Less than $40K          Blue
freq     3951            2329           3551            2643          7162
****************************************
       Gender Education_Level Marital_Status Income_Category Card_Category
count    2628            2628           2628            2628          2628
unique      2               7              4               6             4
top         F        Graduate        Married  Less than $40K          Blue
freq     1406             799           1136             918          2273


In [310]:
print(df['Marital_Status'].value_counts())
print('--'*40)
print(df['Income_Category'].value_counts())
print('--'*40)
print(df['Card_Category'].value_counts())

Married     3551
Single      2876
Divorced     558
Unknown      513
Name: Marital_Status, dtype: int64
--------------------------------------------------------------------------------
Less than $40K    2643
$40K - $60K       1328
$80K - $120K      1132
$60K - $80K       1057
Unknown            820
$120K +            518
Name: Income_Category, dtype: int64
--------------------------------------------------------------------------------
Blue        7162
Silver       295
Gold          38
Platinum       3
Name: Card_Category, dtype: int64


Now let's fill up the missing values: 

In [311]:
df['Customer_Age'].isnull().sum()

10

In [312]:
df['Credit_Limit'].isnull().sum()

1

There are several methods, like filling the missing values with mean or median or mode etc. But the best method in this case is to remove those missing rows since there are only 14 entries and 2. So removing them will not affect our results much.

In [313]:
df = df.dropna()
df_test = df_test.dropna()

In [314]:
print(df.isnull().sum())
print('*'*40)
print(df_test.isnull().sum())

CLIENTNUM                   0
Attrition_Flag              0
Customer_Age                0
Gender                      0
Dependent_count             0
Education_Level             0
Marital_Status              0
Income_Category             0
Card_Category               0
Months_on_book              0
Total_Relationship_Count    0
Months_Inactive_12_mon      0
Contacts_Count_12_mon       0
Credit_Limit                0
Total_Revolving_Bal         0
Trans_Amt_Oct12             0
Trans_Amt_Nov12             0
Trans_Amt_Dec12             0
Trans_Amt_Jan13             0
Trans_Amt_Feb13             0
Trans_Amt_Mar13             0
Trans_Count_Oct12           0
Trans_Count_Nov12           0
Trans_Count_Dec12           0
Trans_Count_Jan13           0
Trans_Count_Feb13           0
Trans_Count_Mar13           0
dtype: int64
****************************************
CLIENTNUM                   0
Attrition_Flag              0
Customer_Age                0
Gender                      0
Dependent_count 

Great now there are no missing values 

Now lets check for the ratio of transactions in final quater of 2012 and first quater of 2013 to check if there has been significant change in the transaction activity.  

In [315]:
#df['SumOfColumns_Q4_12_TransCount'] = df['Trans_Count_Oct12'] + df['Trans_Count_Nov12'] + df['Trans_Count_Dec12']
SumOfColumns_Q4_12_TransCount = df['Trans_Count_Oct12'] + df['Trans_Count_Nov12'] + df['Trans_Count_Dec12']
SumOfColumns_Q4_12_TransCount_test = df_test['Trans_Count_Oct12'] + df_test['Trans_Count_Nov12'] + df_test['Trans_Count_Dec12']

In [316]:
#df['SumOfColumns_Q1_13_TransCount'] = df['Trans_Count_Jan13'] + df['Trans_Count_Feb13'] + df['Trans_Count_Mar13']
SumOfColumns_Q1_13_TransCount = df['Trans_Count_Jan13'] + df['Trans_Count_Feb13'] + df['Trans_Count_Mar13']
SumOfColumns_Q1_13_TransCount_test = df_test['Trans_Count_Jan13'] + df_test['Trans_Count_Feb13'] + df_test['Trans_Count_Mar13']

In [317]:
#df['SumOfColumns_Q4_12_TransAmount'] = df['Trans_Amt_Oct12'] + df['Trans_Amt_Nov12'] + df['Trans_Amt_Dec12']
SumOfColumns_Q4_12_TransAmount = df['Trans_Amt_Oct12'] + df['Trans_Amt_Nov12'] + df['Trans_Amt_Dec12']
SumOfColumns_Q4_12_TransAmount_test = df_test['Trans_Amt_Oct12'] + df_test['Trans_Amt_Nov12'] + df_test['Trans_Amt_Dec12']

In [318]:
#df['SumOfColumns_Q1_13_TransAmount'] = df['Trans_Amt_Jan13'] + df['Trans_Amt_Feb13'] + df['Trans_Amt_Mar13']
SumOfColumns_Q1_13_TransAmount = df['Trans_Amt_Jan13'] + df['Trans_Amt_Feb13'] + df['Trans_Amt_Mar13']
SumOfColumns_Q1_13_TransAmount_test = df_test['Trans_Amt_Jan13'] + df_test['Trans_Amt_Feb13'] + df_test['Trans_Amt_Mar13']

In [319]:
#df['Ratio_TransCount'] = df['SumOfColumns_Q1_13_TransCount']/df['SumOfColumns_Q4_12_TransCount']
df['Ratio_TransCount'] = SumOfColumns_Q1_13_TransCount/SumOfColumns_Q4_12_TransCount
df_test['Ratio_TransCount'] = SumOfColumns_Q1_13_TransCount_test/SumOfColumns_Q4_12_TransCount_test

In [320]:
#df['Ratio_TransAmount'] = df['SumOfColumns_Q1_13_TransAmount']/df['SumOfColumns_Q4_12_TransAmount']
df['Ratio_TransAmount'] = SumOfColumns_Q1_13_TransAmount/SumOfColumns_Q4_12_TransAmount
df_test['Ratio_TransAmount'] = SumOfColumns_Q1_13_TransAmount_test/SumOfColumns_Q4_12_TransAmount_test

In [321]:
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Trans_Amt_Feb13,Trans_Amt_Mar13,Trans_Count_Oct12,Trans_Count_Nov12,Trans_Count_Dec12,Trans_Count_Jan13,Trans_Count_Feb13,Trans_Count_Mar13,Ratio_TransCount,Ratio_TransAmount
0,768805383,0,45.0,M,3,High School,Married,$60K - $80K,Blue,39,...,196,262,0,7,9,12,7,7,1.625000,1.334694
1,818770008,0,49.0,F,5,Graduate,Single,Less than $40K,Blue,44,...,222,238,0,1,6,11,7,8,3.714286,1.541339
2,713982108,0,51.0,M,3,Graduate,Married,$80K - $120K,Blue,36,...,213,182,0,1,5,5,4,5,2.333333,2.594286
3,769911858,0,40.0,F,4,High School,Unknown,Less than $40K,Blue,34,...,285,215,0,4,2,5,2,7,2.333333,1.404517
4,709106358,0,40.0,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,88,279,0,1,7,9,5,6,2.500000,2.175097


In [322]:
df_test.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Trans_Amt_Feb13,Trans_Amt_Mar13,Trans_Count_Oct12,Trans_Count_Nov12,Trans_Count_Dec12,Trans_Count_Jan13,Trans_Count_Feb13,Trans_Count_Mar13,Ratio_TransCount,Ratio_TransAmount
0,772150983,1,48.0,F,3,High School,Unknown,Less than $40K,Blue,37,...,260,415,8,8,10,5,1,8,0.538462,0.662461
1,715128408,0,49.0,F,2,Unknown,Married,Less than $40K,Blue,36,...,316,715,17,10,8,14,9,9,0.914286,0.605301
2,717856983,0,42.0,F,5,Uneducated,Married,Unknown,Blue,29,...,457,570,13,15,23,10,6,9,0.490196,0.605048
3,778231983,0,49.0,F,4,Unknown,Single,$40K - $60K,Blue,39,...,913,774,11,15,13,11,4,11,0.666667,0.806257
4,823908858,0,51.0,M,4,Unknown,Married,$80K - $120K,Blue,47,...,697,586,12,11,18,14,10,16,0.975610,0.794337


In [323]:
Y_train = df['Attrition_Flag']
Y_test = df_test['Attrition_Flag']

In [324]:
df_new = df.drop(['Attrition_Flag'], axis = 1)
df_new_test = df_test.drop(['Attrition_Flag'], axis = 1)

In [325]:
df_new.head(15)

,CLIENTNUM,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,...,Trans_Amt_Feb13,Trans_Amt_Mar13,Trans_Count_Oct12,Trans_Count_Nov12,Trans_Count_Dec12,Trans_Count_Jan13,Trans_Count_Feb13,Trans_Count_Mar13,Ratio_TransCount,Ratio_TransAmount
0,768805383,45.0,M,3,High School,Married,$60K - $80K,Blue,39,5,...,196,262,0,7,9,12,7,7,1.625000,1.334694
1,818770008,49.0,F,5,Graduate,Single,Less than $40K,Blue,44,6,...,222,238,0,1,6,11,7,8,3.714286,1.541339
2,713982108,51.0,M,3,Graduate,Married,$80K - $120K,Blue,36,4,...,213,182,0,1,5,5,4,5,2.333333,2.594286
3,769911858,40.0,F,4,High School,Unknown,Less than $40K,Blue,34,3,...,285,215,0,4,2,5,2,7,2.333333,1.404517
4,709106358,40.0,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,...,88,279,0,1,7,9,5,6,2.500000,2.175097
5,713061558,44.0,M,2,Graduate,Married,$40K - $60K,Blue,36,3,...,55,95,0,1,12,8,1,2,0.846154,1.375546
6,810347208,51.0,M,4,Unknown,Married,$120K +,Gold,46,6,...,228,202,0,8,10,8,2,3,0.722222,1.975391
7,818906208,32.0,M,0,High School,Unknown,$60K - $80K,Silver,27,2,...,329,198,0,4,17,10,2,3,0.714286,2.204167
8,710930508,37.0,M,3,Uneducated,Single,$60K - $80K,Blue,36,5,...,195,246,0,9,2,9,2,2,1.181818,3.354839
9,719661558,48.0,M,2,Graduate,Single,$80K - $120K,Blue,36,6,...,212,169,0,7,10,7,4,4,0.882353,1.523643


Now let's finalize our dataframe from the prescribed calculations, considering numerical dataset: 

In [326]:
df_final = df_new[['Total_Revolving_Bal', 'Ratio_TransCount', 'Months_Inactive_12_mon', 'Total_Relationship_Count', 'Contacts_Count_12_mon']].copy()
df_final_test = df_new_test[['Total_Revolving_Bal', 'Ratio_TransCount', 'Months_Inactive_12_mon', 'Total_Relationship_Count', 'Contacts_Count_12_mon']].copy()

In [327]:
df_final.head()

,Total_Revolving_Bal,Ratio_TransCount,Months_Inactive_12_mon,Total_Relationship_Count,Contacts_Count_12_mon
0,777,1.625000,1,5,3
1,864,3.714286,1,6,2
2,0,2.333333,1,4,0
3,2661,2.333333,4,3,1
4,0,2.500000,1,5,0


In [328]:
df_final_test.head()

,Total_Revolving_Bal,Ratio_TransCount,Months_Inactive_12_mon,Total_Relationship_Count,Contacts_Count_12_mon
0,978,0.538462,2,4,1
1,863,0.914286,3,5,3
2,1267,0.490196,2,3,3
3,1326,0.666667,2,6,3
4,622,0.975610,3,4,2


In [329]:
X_train = df_final
X_test = df_final_test

In [330]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

99.76

In [332]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

99.75

In [331]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7487 entries, 0 to 7497
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CLIENTNUM                 7487 non-null   int64  
 1   Customer_Age              7487 non-null   float64
 2   Gender                    7487 non-null   object 
 3   Dependent_count           7487 non-null   int64  
 4   Education_Level           7487 non-null   object 
 5   Marital_Status            7487 non-null   object 
 6   Income_Category           7487 non-null   object 
 7   Card_Category             7487 non-null   object 
 8   Months_on_book            7487 non-null   int64  
 9   Total_Relationship_Count  7487 non-null   int64  
 10  Months_Inactive_12_mon    7487 non-null   int64  
 11  Contacts_Count_12_mon     7487 non-null   int64  
 12  Credit_Limit              7487 non-null   float64
 13  Total_Revolving_Bal       7487 non-null   int64  
 14  Trans_Am